In [1]:
import os
import glob
import pickle
import numpy as np
import os
import math
import numpy as np
import itertools
import cv2
import pickle
import copy
import torch
import open3d as o3d
import random
from pathlib import Path
from shutil import copyfile
import copy

In [2]:
def rotate_points_along_z(points, angle):
    """
    Args:
        points: (B, N, 3 + C)
        angle: (B), angle along z-axis, angle increases x ==> y
    Returns:

    """
    points, is_numpy = check_numpy_to_torch(points)
    angle, _ = check_numpy_to_torch(angle)

    cosa = torch.cos(angle)
    sina = torch.sin(angle)
    zeros = angle.new_zeros(points.shape[0])
    ones = angle.new_ones(points.shape[0])
    rot_matrix = torch.stack((
        cosa, sina, zeros,
        -sina, cosa, zeros,
        zeros, zeros, ones
    ), dim=1).view(-1, 3, 3).float()
    points_rot = torch.matmul(points[:, :, 0:3], rot_matrix)
    points_rot = torch.cat((points_rot, points[:, :, 3:]), dim=-1)
    return points_rot.numpy() if is_numpy else points_rot


def check_numpy_to_torch(x):
    if isinstance(x, np.ndarray):
        return torch.from_numpy(x).float(), True
    return x, False


def boxes_to_corners_3d(boxes3d):
    """
        7 -------- 4
       /|         /|
      6 -------- 5 .
      | |        | |
      . 3 -------- 0
      |/         |/
      2 -------- 1
    Args:
        boxes3d:  (N, 7) [x, y, z, dx, dy, dz, heading], (x, y, z) is the box center

    Returns:
    """
    boxes3d, is_numpy = check_numpy_to_torch(boxes3d)
    template = boxes3d.new_tensor((
        [1, 1, -1], [1, -1, -1], [-1, -1, -1], [-1, 1, -1],
        [1, 1, 1], [1, -1, 1], [-1, -1, 1], [-1, 1, 1],
    )) / 2

    corners3d = boxes3d[:, None, 3:6].repeat(1, 8, 1) * template[None, :, :]
    corners3d = rotate_points_along_z(corners3d.view(-1, 8, 3), boxes3d[:, 6]).view(-1, 8, 3)
    corners3d += boxes3d[:, None, 0:3]

    return corners3d.numpy() if is_numpy else corners3d

def show_laser_with_box(
        pcds, pcds_color=None,
        box3ds=None, box3d_color=[255, 0, 0],
        categorys=None,
        pred_box3ds=None, pred_box3d_color=[0, 0, 255],
        pred_categorys=None,
        x_range=(-40, 62.4), y_range=(-40, 40), z_range=(-3, 8),
        name='laser'
):
    '''

    :param pcds: numpy float32 array (n, c)
    :param pcds_color: pcds_color uint8 array (n, 3) denotes the color of each point 0~255
    :param box3ds: box3d float32 numpy array; [x,y,z,l,w,h,heading]
    :param box3d_color:
    :param categorys:
    :param pred_box3ds:
    :param pred_box3d_color:
    :param pred_categorys:
    :param x_range:
    :param y_range:
    :param z_range:
    :param name:
    :return:
    '''
    import open3d as o3d
    pcd_l = o3d.geometry.PointCloud()
    pcd_l.points = o3d.utility.Vector3dVector(pcds[:, :3])
    if pcds_color is not None:
        pcd_l.colors = o3d.utility.Vector3dVector(pcds_color)

    lines = [[0, 1], [1, 5], [5, 4], [4, 0],
             [3, 2], [2, 6], [6, 7], [7, 3],
             [0, 3], [4, 7], [1, 2], [5, 6]]

    line_sets = []
    if box3ds is not None:
        box3d_corners = boxes_to_corners_3d(box3ds)
        for box3d_corner in box3d_corners:
            line_set = o3d.geometry.LineSet()
            line_set.points = o3d.utility.Vector3dVector(box3d_corner)

            line_set.lines = o3d.utility.Vector2iVector(lines)
            line_set.colors = o3d.utility.Vector3dVector([box3d_color for i in range(len(lines))])

            line_sets.append(line_set)

    if pred_box3ds is not None:
        pred_box3d_corners = boxes_to_corners_3d(pred_box3ds)
        for box3d_corner in pred_box3d_corners:
            line_set = o3d.geometry.LineSet()
            line_set.points = o3d.utility.Vector3dVector(box3d_corner)

            line_set.lines = o3d.utility.Vector2iVector(lines)
            line_set.colors = o3d.utility.Vector3dVector([pred_box3d_color for i in range(len(lines))])

            line_sets.append(line_set)

    line_set = o3d.geometry.LineSet()
    corners = []
    for z in z_range:
        corners.append([x_range[0], y_range[0], z])
        corners.append([x_range[0], y_range[1], z])
        corners.append([x_range[1], y_range[1], z])
        corners.append([x_range[1], y_range[0], z])

    line_set.points = o3d.utility.Vector3dVector(corners)
    colors = [[0, 0, 0] for i in range(len(lines))]
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)
    line_sets.append(line_set)
    o3d.visualization.draw_geometries([*line_sets, pcd_l], window_name=name)
    
    
def show_image_with_box(image, boxes, categorys, output_dir, name=None, thickness=1, font_scale=0.3):
    image = copy.deepcopy(image)
    image = np.ascontiguousarray(image[:, :, ::-1])  # RGB -> BGR
    boxes = copy.deepcopy(boxes)
    color = (0, 255, 255)
    for i in range(len(boxes)):
        # extract the bounding box coordinates
        cx, cy = boxes[i][0], boxes[i][1]
        w, h = boxes[i][2], boxes[i][3]
        xmin, ymin = cx - w / 2, cy - h / 2
        xmin, ymin, w, h = map(int, [xmin, ymin, w, h])
        category_index = categorys[i]
        category = str(category_index)

        # draw a bounding box rectangle and label on the image
        cv2.rectangle(image, (xmin, ymin), (xmin + w, ymin + h), color=color, thickness=thickness)
        confidence = 1.0
        text = f"{category}: {confidence:.2f}"

        # calculate text width & height to draw the transparent boxes as background of the text
        (text_width, text_height) = \
            cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, fontScale=font_scale, thickness=thickness)[0]
        text_offset_x = xmin
        text_offset_y = ymin - 5
        box_coords = ((text_offset_x, text_offset_y), (text_offset_x + text_width + 2, text_offset_y - text_height))

        overlay = image.copy()
        cv2.rectangle(overlay, box_coords[0], box_coords[1], color=color, thickness=cv2.FILLED)
        # add opacity (transparency to the box)
        image = cv2.addWeighted(overlay, 0.6, image, 0.4, 0)
        # now put the text (label: confidence %)
        cv2.putText(image, text, (xmin, ymin - 5), cv2.FONT_HERSHEY_SIMPLEX,
                    fontScale=font_scale, color=(0, 0, 0), thickness=thickness)
    print(os.path.join(output_dir, "image_detection_result_{}.jpg".format(name)))
    cv2.imwrite(os.path.join(output_dir, "image_detection_result_{}.jpg".format(name)), image)


In [4]:
'''
how to save 

import pickle
frame_id = batch_dict['frame_id'][0]
pickle.dump(batch_dict['points'].cpu().numpy(), open('../output/bins/4/points.bin', 'wb'))
pickle.dump(batch_dict['gt_boxes'][0].cpu().numpy(), open('../output/bins/4/gt_boxes.bin', 'wb'))
pickle.dump(batch_dict['rois'][0].cpu().numpy(), open('../output/bins/4/pred_boxes.bin', 'wb'))
'''

input_dir = '/Users/jiangtao.jiangtao/tmp/AAAAAbins/'
dirs = os.listdir(input_dir)
if '.DS_Store' in dirs:
    dirs.remove('.DS_Store')

for _dir in dirs:
    point_file = os.path.join(input_dir, _dir, 'points.bin')
    preds_file = os.path.join(input_dir, _dir, 'pred_boxes_.bin')
    gt_file = os.path.join(input_dir, _dir, 'gt_boxes.bin')

    with open(point_file, 'rb') as f:
        points = pickle.load(f)
        points = points[:, 1:]
    with open(gt_file, 'rb') as f:
        annos = pickle.load(f)
        annos = annos[:, :-1]
        print(annos.shape)
    with open(preds_file, 'rb') as f:
        preds = pickle.load(f)

    show_laser_with_box(pcds=points, box3ds=annos, pred_box3ds=preds, name='')

(11, 7)


In [ ]:
nums = 10
points_dir = '/home/jiangtao.jiangtao/code/opensource/OpenPCDet/' + 'data/waymo/validation/sample/laser/' 
anno_dir = '/home/jiangtao.jiangtao/code/opensource/OpenPCDet/' + 'data/waymo/validation/sample/label/' 
pred_dir = '/home/jiangtao.jiangtao/code/opensource/OpenPCDet/output/waymo_models/pv_rcnn/' + '2020-07-21-01-00/eval/epoch_15/testing/default/final_result/data/'

output_dir = '/home/jiangtao.jiangtao/code/opensource/OpenPCDet/output/sampling/'

output_dir = Path(output_dir)

points_dir = Path(points_dir)
anno_dir = Path(anno_dir)
pred_dir = Path(pred_dir)

output_dir.mkdir(parents=True, exist_ok=True)
(output_dir/'laser').mkdir(parents=True, exist_ok=True)
(output_dir/'label').mkdir(parents=True, exist_ok=True)
(output_dir/'pred').mkdir(parents=True, exist_ok=True)

files = os.listdir(pred_dir)
sampling = random.choices(files, k=nums)
for sample in sampling:
    src = points_dir / sample
    dst = output_dir/'laser' / sample
    copyfile(src, dst)
    
    src = anno_dir / sample
    dst = output_dir/'label' / sample
    copyfile(src, dst)
    
    src = pred_dir / sample
    dst = output_dir/'pred' / sample
    copyfile(src, dst)

In [ ]:
input_dir = '/Users/jiangtao.jiangtao/tmp/AAsampling/'
files = glob.glob(input_dir + 'laser/*bin')
files = [os.path.basename(file) for file in files]

for file in files:
    print(file)
    
    point_file = os.path.join(input_dir, 'laser', file)
    anno_file = os.path.join(input_dir, 'label', file)
    preds_file = os.path.join(input_dir, 'pred', file)

    with open(point_file, 'rb') as f:
        points = pickle.load(f)
    with open(preds_file, 'rb') as f:
        preds = pickle.load(f)
        boxes = [[anno.box.center_x,anno.box.center_y,anno.box.center_z,anno.box.length,anno.box.width,anno.box.height,anno.box.heading] for anno in preds]
        boxes = np.asarray(boxes)

    with open(anno_file, 'rb') as f:
        annos = pickle.load(f)
        annos_boxes = [[anno.box.center_x,anno.box.center_y,anno.box.center_z,anno.box.length,anno.box.width,anno.box.height,anno.box.heading] for anno in annos]
        annos_boxes = np.asarray(annos_boxes)
        point_threshold = 20
        selected_index = np.asarray([True if anno.num_lidar_points_in_box > point_threshold else False for anno in annos])
        ignored_index = np.asarray([True if anno.num_lidar_points_in_box < point_threshold else False for anno in annos])
        selected_annos = annos_boxes[selected_index]
        ignored_annos = annos_boxes[ignored_index]
#         annos_boxes = annos_boxes[selected_index]

    show_laser_with_box(pcds=points, box3ds=ignored_annos, pred_box3ds=selected_annos, name=str(file), 
                        x_range=(-80, 80), y_range=(-80, 80), z_range=(-2.5, 2.5),)

In [ ]:
input_dir = '/Users/jiangtao.jiangtao/tmp/Adebug/'
files = glob.glob(input_dir + 'laser/*bin')
files = [os.path.basename(file) for file in files]
for file in files:
    print(file)

    point_file = os.path.join(input_dir, 'laser', file)
    anno_file = os.path.join(input_dir, 'label', file)

    with open(point_file, 'rb') as f:
        points = pickle.load(f)

    with open(anno_file, 'rb') as f:
        annos = pickle.load(f)
        annos_boxes = [
            [anno.box.center_x, anno.box.center_y, anno.box.center_z, anno.box.length, anno.box.width, anno.box.height,
             anno.box.heading] for anno in annos]
        annos_boxes = np.asarray(annos_boxes)
        point_threshold = 20
        selected_index = np.asarray(
            [True if anno.num_lidar_points_in_box > point_threshold else False for anno in annos])
        ignored_index = np.asarray(
            [True if anno.num_lidar_points_in_box < point_threshold else False for anno in annos])
        selected_annos = annos_boxes[selected_index]
        ignored_annos = annos_boxes[ignored_index]
    #         annos_boxes = annos_boxes[selected_index]
    pcds_color = np.zeros((points.shape[0], 3), dtype=np.uint)

    # 按照类别染色
    #     for label in np.unique(points[:,5]):
    #         index = points[:,5] == label
    #         if label == 1: # car  红色
    #             pcds_color[index] = [255,0,0]
    #         elif label == 2: # pedestrian 绿色
    #             pcds_color[index] = [0,255,0]
    #         elif label == 3: # cyclist 蓝色
    #             pcds_color[index] = [0,0,255]
    #         elif label == -1: # background  黄色
    #             pcds_color[index] = [255,255,0]
    # 按照texture染色
    # pcds_color = points[:, 6:9] / 256

    # 按照geometric染色
    # annos_boxes = annos_boxes[43:44,]
    # annos_boxes = annos_boxes[129:130,]
    # x = points[:, 0]
    # y = points[:, 1]
    # z = points[:, 2]
    # index = (x > annos_boxes[0][0] - annos_boxes[0][3]/2) & (x < annos_boxes[0][0] + annos_boxes[0][3]/2) \
    #       & (y > annos_boxes[0][1] - annos_boxes[0][4]/2) & (y < annos_boxes[0][1] + annos_boxes[0][4]/2) & \
    #         (z > annos_boxes[0][2] - annos_boxes[0][5]/2) & (z < annos_boxes[0][2] + annos_boxes[0][5]/2)
    # points = points[index]

    pcds_color = np.zeros(shape=(points.shape[0], 3), dtype=np.float)
    pcds_color[:, 0] = points[:, 5+1+3]
    pcds_color[:, 1] = points[:, 5+1+3+1]
    # pcds_color[:, 0] = points[:, 5+1+3+1+1]
    show_laser_with_box(pcds=points, pcds_color=pcds_color, box3ds=annos_boxes, pred_box3ds=None, name=str(file),
                        x_range=(-80, 80), y_range=(-80, 80), z_range=(-2.5, 2.5), )

In [ ]:
input_dir = '/Users/jiangtao.jiangtao/tmp/Adebug/'
files = glob.glob(input_dir + 'image/*bin')
files = [os.path.basename(file) for file in files]
for file in files:
    boxes = pickle.load(open(os.path.join(input_dir,'image_label',file),'rb'))
    image = pickle.load(open(os.path.join(input_dir,'image',file),'rb'))
    show_image_with_box(image, boxes, np.asarray([1 for i in range(len(boxes))]), output_dir='output/', name=file, thickness=1, font_scale=0.3)